# Imports & Settings


In [2]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from deep_translator import GoogleTranslator
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [77]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})

/tmp/ipykernel_25152/1699356385.py:9: DtypeWarning: Columns (13,266) have mixed types. Specify dtype option on import or set low_memory=False.
  prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})


# Preparation

In [78]:
data = prep.copy()

In [42]:
text_features = ['name', 'commentary', 'text', 'legend', 'initial_info', 'additional_text']

In [79]:
data[text_features] = data[text_features].fillna('')

In [49]:
def collect_text(item):
    return ' '.join(item[text_features]).strip()

In [80]:
data['text_features'] = data.apply(lambda item: collect_text(item),axis=1)

In [81]:
data[['name', 'commentary', 'text', 'legend', 'initial_info', 'additional_text', 'text_features']].head(2)

,name,commentary,text,legend,initial_info,additional_text,text_features
id,,,,,,,
232170,"Kuno Areng, Bremerhaven Festwoche medal",,,,Festwoche - Breemenhaven,KUTTER ASTARTE -SCHIFFERGILDE BREMENHAVEN E.V.,"Kuno Areng, Bremerhaven Festwoche medal Festwoche - Breemenhaven KUTTER ASTARTE -SCHIFFERGILDE BREMENHAVEN E.V."
2251378,"Photo-Villem Kapp, photo with dedication to Armilde M, 1937",,,"Photos from the collection of Villem Kapi and Juhan Aavik\ndesse, purchased in 2013",,,"Photo-Villem Kapp, photo with dedication to Armilde M, 1937 Photos from the collection of Villem Kapi and Juhan Aavik\ndesse, purchased in 2013"


In [82]:
data.to_csv('data/prep.csv')

# check for type string

In [57]:
data = prep.copy()

In [59]:
data.text_features.info()

<class 'pandas.core.series.Series'>
Int64Index: 20000 entries, 232170 to 2781747
Series name: text_features
Non-Null Count  Dtype 
--------------  ----- 
14900 non-null  object
dtypes: object(1)
memory usage: 312.5+ KB


In [75]:
types = data.type.unique()
# remove nan
types = types[:-1]

In [73]:
counter = 0
true_counter = 0
for i, item in data.iterrows():
    # train ds
    if type(item.type) == str:
        local_counter = 0
        for type in types:
            if type in item.text_features:
                local_counter += 1
                pred = type
        if local_counter > 1:
            print('!')
        if local_counter == 1:
            counter += 1
            if pred == item.type:
                true_counter += 1

TypeError: 'in <string>' requires string as left operand, not float

In [72]:
a = 0
for i, item in data.iterrows():
    print(type(item.type))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [76]:
types

array(['medal', 'photo', 'photo negative', 'a letter', 'coin',
       'label/sign', 'script, song/vocal music', 'sheet music', 'slide',
       'archaeological find', 'book', 'plan', 'postcard', 'there/imprint',
       'poster', 'design/drawing/sketch', 'audio recording', 'manuscript',
       'document', 'caricature', 'graphics', 'telegram',
       'manuscript, musical composition', 'notes', 'in small print',
       'invitation', 'packaging', 'paper', 'manuscript, sheet music',
       'magazines', 'album', 'in the country', 'digital image',
       'photo, postcard', 'photographic material', 'drawing', 'crate/box',
       'letter of honor/honorary address', 'calendar', 'tape/ribbon',
       'bag', 'newspaper', 'dish/vessel', 'musical instrument',
       'letter, postcard', 'sculpture', 'printed notes', 'there',
       'a suit', 'certificate', 'sheet/linen', 'folder/booklet', 'jewel',
       'doll', 'photographic negative, photographic negative'],
      dtype=object)